## Converting dates to a consistent format
This demo shows how you can take a date field that may be in a variety of formats, have bad data, etc. and convert it to a field with a consistent format and NA for any bad values.

Dates are a challenging problem because there are so many different ways to write them, and they can often be ambiguous. Some of the cases that are testing in the demo dataset:
* The month may be specified with a month name (January) or a month number (1)
* Numbers may be specified as digits (15) or words (fifteen)
* The century is often implied: 4/17/94 implies the year is 1994, while 9/20/13 implies the year is 2013
* The US and Europe have different conventions for the order of day and month, so 04/10/2015 is April 10 2015 in the US and 4 October 2015 in Europe

In [1]:
import os
import pandas as pd

In [2]:
import sys
sys.path.append('../..')
import openai_data_tools as dt

In [3]:
examples = [
       {'item': 'May 5, 1985', 'target': '1985-05-05'}
]

In [6]:
dates = pd.read_csv('dates.csv', dtype=str, keep_default_na=False)

Here is our test data set: Item is the input to the model, target is the desired output.

In [7]:
dates

,item,target,notes
0,4-19-2008,2008-04-19,
1,8/17/2019,2019-08-17,
2,1.28.56,1956-01-28,Has to infer that we mean 1956 and not 2056
3,10.23.22,2022-10-23,Has to infer that we mean 2022 and not 1922
4,February 14th 2010,2010-02-14,
5,"Dec 12, 1995",1995-12-12,
6,23 March 1974,1974-03-23,
7,7/9/80,1980-07-09,Ambiguous: Should this date be interpreted wit...
8,14/19/93,NA,This is not a valid date for either convention
9,2/15,NA,This is an invalid date because no year is spe...


In [8]:
processor = dt.DataProcessor(
    api_key=os.getenv("OPENAI_API_KEY"),
    model = 'gpt-3.5-turbo', 
    instructions = "You will be provided with dates in various formats. For each date, convert it to the format YYYY-MM-DD. If it is not a valid date, return 'NA'."
)

In [9]:
output = processor.process(dates['item'])

Progress: 100%


Let's use a Scorer object to see how the model did.

In [10]:
scorer = dt.Scorer(output, dates['target'])

This is the proportion of items where the output matched the target.

In [11]:
scorer.accuracy()

0.9230769230769231

This lets us see the specific items where the output didn't match a target (indicated by 0)

In [12]:
scorer.score()

[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

This lets us look at the input, the target output, and the actual output side-by-side

In [13]:
pd.DataFrame({'item': dates['item'], 'target': dates['target'], 'output': output})

,item,target,output
0,4-19-2008,2008-04-19,2008-04-19
1,8/17/2019,2019-08-17,2019-08-17
2,1.28.56,1956-01-28,NA
3,10.23.22,2022-10-23,2022-10-23
4,February 14th 2010,2010-02-14,2010-02-14
5,"Dec 12, 1995",1995-12-12,1995-12-12
6,23 March 1974,1974-03-23,1974-03-23
7,7/9/80,1980-07-09,1980-07-09
8,14/19/93,NA,NA
9,2/15,NA,NA
